# Divide tweets by following
User must follow at least 3 political influencers, at least 80% of which must be left or right wing for the user to receive a label

In [33]:
import pickle

In [34]:
with open(r"leftdict.pkl", "rb") as input_file:
    leftdict = pickle.load(input_file)
    
with open(r"rightdict.pkl", "rb") as input_file:
    rightdict = pickle.load(input_file)

In [49]:
import tweepy
import configparser
import datetime
from tqdm import tqdm

In [42]:
config = configparser.ConfigParser()
config.read('../twitter_api/config.ini')

['../twitter_api/config.ini']

In [43]:
#loading the keys
api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']
access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

In [44]:
#authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit = True)

In [46]:
#top 177 left wing influencer IDs

left_ids = []

for handle in tqdm(list(leftdict.keys())[:177]):
    
    id = api.get_user(screen_name=handle).id_str
    left_ids.append(id)

In [50]:
#top 177 right wing influencer IDs

right_ids = []

for handle in tqdm(list(rightdict.keys())[:177]):
    
    id = api.get_user(screen_name=handle).id_str
    right_ids.append(id)

100%|████████████████████████████████████████| 177/177 [00:50<00:00,  3.49it/s]


In [59]:
intersect = list(set(left_ids) & set(right_ids))

In [60]:
#remove intersect between left and right
left_ids = [x for x in left_ids if x not in intersect]
right_ids = [x for x in right_ids if x not in intersect]

#### testing on example tweets

In [3]:
with open(r"MVP/tweets042522.pkl", "rb") as input_file:
    tweets042522 = pickle.load(input_file)

In [4]:
tweets042522

,Datetime,Tweet Id,Text,Username,Keyword,CleanedText,VADER_Compound
0,2022-04-26 23:59:51+00:00,1519103999030579200,"You know, instead of all the tweets of ""Elon ...",whatisKhaiki,#TwitterTakeOver,"You know, instead of all the tweets of ""Elon M...",-0.6486
1,2022-04-26 23:59:16+00:00,1519103853324611584,Pssstttt I have a secret …. Turns out @elonmus...,CoshJoulter,#TwitterTakeOver,Pssstttt I have a secret .... Turns out @elonm...,-0.2500
2,2022-04-26 23:59:04+00:00,1519103801571135489,Twitter take over😂\n\n@elonmusk the kung fu is...,MrSotAman1,#TwitterTakeOver,Twitter take over😂 @elonmusk the kung fu is st...,0.1280
3,2022-04-26 23:58:56+00:00,1519103769321033728,To the people who are so worked up about how @...,soitweetyeah76,#TwitterTakeOver,To the people who are so worked up about how @...,0.4215
4,2022-04-26 23:58:39+00:00,1519103694809100288,President Trump's next tweet; the world's most...,dialecticum,#TwitterTakeOver,President Trump's next tweet; the world's most...,-0.1010
...,...,...,...,...,...,...,...
4723,2022-04-26 23:58:43+00:00,1519103713125806081,Mr Elon Musk for president 🤔🤔 https://t.co/BdL...,consuleduc,Elon Musk,Mr Elon Musk for president 🤔🤔 <URL>,0.0000
4724,2022-04-26 23:58:43+00:00,1519103712098201601,"""You could run a Hyperloop through the vacuum ...",heymrbrad,Elon Musk,"""You could run a Hyperloop through the vacuum ...",0.6808
4725,2022-04-26 23:58:42+00:00,1519103707954176000,Elon Musk would have the world trade excessive...,LGKatzPhD,Elon Musk,Elon Musk would have the world trade excessive...,0.0000
4726,2022-04-26 23:58:41+00:00,1519103706674958336,If Elon #Musk starts charging for your #Twitte...,BMW_Borger,Elon Musk,If Elon #Musk starts charging for your #Twitte...,0.5484


#### Trying scweet for scraping user following
https://github.com/Altimis/Scweet

In [73]:
#!pip install Scweet==1.8

In [74]:
#!pip install certifi python-dotenv chromedriver-autoinstaller geckodriver-autoinstaller urllib3

In [76]:
from Scweet.scweet import scrape
from Scweet.user import get_user_information, get_users_following, get_users_followers

In [84]:
users=tweets042522.Username.unique()[:2]
env_path='.env'
following_info=get_users_following(users=users, env=env_path, verbose=0, headless=True, wait=2, limit=50, file_path=None)

Scraping on headless mode.
Crawling whatisKhaiki following
Found 16 following
Found 36 following
Found 50 following
Crawling CoshJoulter following
Found 17 following
Found 36 following
Found 50 following
file saved in outputs/whatisKhaiki_CoshJoulter_following.json


In [71]:
outdict={}
for user in tqdm(tweets042522.Username.unique()[:100]):
    friendlist=api.get_friend_ids(screen_name=user)
    left_intersect = list(set(friendlist) & set(left_ids))
    right_intersect = list(set(friendlist) & set(right_ids))
    totalcount = len(left_intersect + right_intersect)
    if totalcount > 3 and (len(left_intersect)/totalcount) > 0.8:
        outdict[user] = 'Left'
    elif totalcount > 3 and (len(right_intersect)/totalcount) > 0.8:
        outdict[user] = 'Right'
    else:
        outdict[user] = 'None'

 17%|██████▋                                | 17/100 [14:34<1:11:10, 51.45s/it]


NotFound: 404 Not Found
34 - Sorry, that page does not exist.

In [72]:
outdict

{'whatisKhaiki': 'None',
 'CoshJoulter': 'None',
 'MrSotAman1': 'None',
 'soitweetyeah76': 'None',
 'dialecticum': 'None',
 'Nagerharish': 'None',
 'marmotadelaeco': 'None',
 'copycat21c': 'None',
 'TyMorrell': 'None',
 '1Brightway': 'None',
 'nudge2409': 'None',
 'gnickturner': 'None',
 'realBlakeBeye': 'None',
 'KKoomara': 'None',
 'g1patnaik': 'None',
 'GenDisdain': 'None',
 'PUREWHITEDECO': 'None'}